# Selection of Calibration Windows for Day-Ahead Electricity Price Forecasting
## by Grzegorz Marcjasz, Tomasz Serafin and Rafał Weron 

In [2]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

In [3]:
df_day_ahead_epex = pd.read_csv('Day_Ahead_Epex.csv')
df_NP2018 = pd.read_csv('NP2018.csv')
df_PJM = pd.read_csv('PJM.csv')

### 2. Methodology

#### 2.1. Preliminaries

In [5]:
def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

In [ ]:
# Assuming array of prices
prices = np.array([...])

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Steps for model training and forecasting go here

# Assuming forecasts is your array of forecasts
forecasts = np.array([...])

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

#### 2.2. Expert Models

##### ARX1

In [7]:
import statsmodels.api as sm

def prepare_data_arx1(data):
    # Prepare the data for the ARX1 model
    # This includes creating the lagged variables and the dummy variables for the weekdays
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['load_forecast'] = data['load'].shift(24)
    data = pd.get_dummies(data, columns=['weekday']) # create dummy variables for the weekdays
    data = data.dropna()  # drop missing values
    return data

def fit_arx1(data):
    # Fit the ARX1 model
    # The dependent variable is 'X'
    # The independent variables are the lagged variables, the load forecast, and the dummy variables
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'load_forecast', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog) # Ordinary Least Squares (OLS) model
    results = model.fit()
    return results # results of the model fitting

##### AR1

In [8]:
def prepare_data_ar1(data):
    # Prepare the data for the AR1 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_ar1(data):
    # Fit the AR1 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [ ]:
# Prepare the data
data = prepare_data_ar1(data)

# Fit the AR1 model
results = fit_ar1(data)

# Print the model summary
print(results.summary())

##### ARX2

In [9]:
def prepare_data_arx2(data):
    # Prepare the data for the ARX2 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['X_max'] = data['X'].rolling(window=24).max().shift(24)
    data['X_last'] = data['X'].shift(24)
    data['load_forecast'] = data['load'].shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_arx2(data):
    # Fit the ARX2 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'X_max', 'X_last', 'load_forecast', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [10]:
# Prepare the data
data = prepare_data_arx2(data)

# Fit the ARX2 model
results = fit_arx2(data)

# Print the model summary
print(results.summary())

NameError: name 'data' is not defined

##### AR2

In [ ]:
def prepare_data_ar2(data):
    # Prepare the data for the AR2 model
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['X_max'] = data['X'].rolling(window=24).max().shift(24)
    data['X_last'] = data['X'].shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_ar2(data):
    # Fit the AR2 model
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'X_max', 'X_last', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results

In [ ]:
# Prepare the data
data = prepare_data_ar2(data)

# Fit the AR2 model
results = fit_ar2(data)

# Print the model summary
print(results.summary())